In [ ]:
!cp /kaggle/input/gdcm-conda-install/gdcm.tar .
!tar -xvzf gdcm.tar
!conda install --offline ./gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2
!rm -rf ./gdcm.tar
# !git clone https://github.com/lessw2020/Ranger-Deep-Learning-Optimizer.git
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
# sys.path.append('./Ranger-Deep-Learning-Optimizer/ranger')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.autonotebook import tqdm
from pprint import pprint
from datetime import datetime
import os, sys, cv2, glob, random ,ast, warnings, time
warnings.filterwarnings('ignore')

import timm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts
from torch.optim import Adam, SGD, AdamW

# import pytorch_lightning as pl
# import torchmetrics
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import albumentations as A
from albumentations.pytorch import ToTensorV2

# from ranger import Ranger  # this is from ranger.py
# from ranger913A import RangerVA  # this is from ranger913A.py
# from rangerqh import RangerQH  # this is from rangerqh.py

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
sys.path.append("../input/iterative-stratification/iterative-stratification-master")
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
model_names = timm.list_models()
# pprint(model_names)

# CFG

In [ ]:
BATCH_SIZE = 16
VAL_BATCH_SIZE = 32
EPOCHS = 5 
IMG_SIZE = 512
if BATCH_SIZE == 8:
    ITER_FREQ = 300
else:
    ITER_FREQ = 90
NUM_WORKERS = 8
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
SEED = 1111
N_FOLDS = 5
START_FOLD = 3

MODEL_PATH = None
MODEL_ARCH = 'tf_efficientnet_b5_ns' # tf_efficientnet_b4_ns, tf_efficientnet_b6_ns, resnext50_32x4d, seresnet152d

LR = 1e-4
MIN_LR = 1e-6 # CosineAnnealingWarmRestarts
WEIGHT_DECAY = 1e-6
MOMENTUM = 0.9
T_0 = EPOCHS # CosineAnnealingWarmRestarts
MAX_NORM = 1000
T_MAX = 5
ITERS_TO_ACCUMULATE = 1

BASE_OPTIMIZER = SGD #for Ranger
OPTIMIZER = 'Adam' # Ranger, AdamW, AdamP, SGD

SCHEDULER = 'CosineAnnealingWarmRestarts' # ReduceLROnPlateau, CosineAnnealingLR, CosineAnnealingWarmRestarts, OneCycleLR
SCHEDULER_UPDATE = 'epoch' # batch

TR_CRITERION = 'BCE'
VL_CRITERION = 'BCE' # CrossEntropyLoss
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# avg_prec = torchmetrics.AveragePrecision(num_classes = 4)
LABELS = ['Negative for Pneumonia','Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance']

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.val = 0
        self.sum = 0
        self.avg = 0
        self.count = 0
        
    def update(self, val, n=1):
        self.val = val
        self.sum += val*n
        self.count += n
        self.avg = self.sum / self.count
        
def seed_torch(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch(SEED)

def macro_multilabel_auc(label, pred):
    aucs = []
    for i in range(len(LABELS)):
        aucs.append(roc_auc_score(label[:, i], pred[:, i]))
    print(np.round(aucs, 4))
    return np.mean(aucs)

# Dataset

In [ ]:
def preprocess_df(train = True):
    if train:
        df_image = pd.read_csv("../input/siim-covid19-detection/train_study_level.csv")
        df_det = pd.read_csv("../input/siim-covid19-detection/train_image_level.csv")
        df_image['StudyInstanceUID'] = df_image['id'].apply(lambda x : x[:-6])
        df = df_det.merge(df_image, on='StudyInstanceUID')
        path = []
        TRAIN_DIR = "../input/siim-covid19-detection/train/"
        for instance_id in tqdm(df['StudyInstanceUID']):
            path.append(glob.glob(os.path.join(TRAIN_DIR, instance_id +"/*/*"))[0])
        df['path'] = path

        df = df.drop(['id_x', 'id_y'], axis=1)
        return df
    
    else:
        df= pd.read_csv("../input/siim-covid19-detection/sample_submission.csv")
        study_indices = df['id'].apply(lambda x : x[-6:])
        STUDY_LEN =0
        for i in range(len(study_indices)):
            if study_indices[i] == '_image':
                STUDY_LEN = i
                break
            
        df['StudyInstanceUID'] = df['id'].apply(lambda x : x[:-6])
        df = df.iloc[:STUDY_LEN,:]
        path = []
        TEST_DIR = "../input/siim-covid19-detection/test"
        for instance_id in tqdm(df['StudyInstanceUID']):
            path.append(glob.glob(os.path.join(TEST_DIR, instance_id +"/*/*"))[0])
        df['path'] = path

        return df,STUDY_LEN

In [ ]:
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

In [ ]:
#Stratified KFold
def stratifiedKFold(df,num_folds,random_state):
    y = df [['Negative for Pneumonia','Typical Appearance', 
            'Indeterminate Appearance', 'Atypical Appearance']]
    df['fold'] = 0
    #split data
    mskf = MultilabelStratifiedKFold(n_splits=num_folds, shuffle= True, random_state=random_state)
    for i, (_, test_index) in enumerate(mskf.split(df, y)):
        df.iloc[test_index, -1] = i
    return df

df = preprocess_df(train = True)

In [ ]:
class SIIM_COVID(Dataset):
    def __init__(self,df,
                 train = True,
                 transforms=None,
                 IMG_SIZE = 256
                ):
        self.imageList = df['path'].values
        self.transform = None
        if transforms is None:
            self.transform = A.Compose([
                                            A.Resize(IMG_SIZE,IMG_SIZE),
                                            A.Normalize(
                                                mean=[0.485, 0.456, 0.406],
                                                std=[0.229, 0.224, 0.225],
                                            ),
                                            ToTensorV2()
                                        ])
        else:
            self.transform = transforms
        self.train  = train    
        if self.train == True:
            self.labels = df[LABELS].values
    
    def __len__(self):
        return len(self.imageList)
    
    def __getitem__(self,idx):
        file_path = self.imageList[idx]
        img = dicom2array(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image = self.transform(image=img)
        if self.train == True:
            image = image['image']
            label = self.labels[idx]
            return image, label
        else:
            return image

In [ ]:
def get_transform(*, train=True):
    
    if train:
        return A.Compose([
            A.RandomResizedCrop(IMG_SIZE, IMG_SIZE, scale=(0.85, 1.0)),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.2, brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2)),
            A.HueSaturationValue(p=0.2, hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2),
            A.ShiftScaleRotate(p=0.2, shift_limit=0.0625, scale_limit=0.2, rotate_limit=20),
            A.Normalize(mean=MEAN, std=STD),
            ToTensorV2(), # commented now will convert to torch tensors later.
        ])
    else:
        return A.Compose([
#             A.CenterCrop(IMG_SIZE, IMG_SIZE),
            A.Resize(IMG_SIZE, IMG_SIZE),
            A.Normalize(mean=MEAN, std=STD, max_pixel_value=255.0, p=1.0),
            ToTensorV2(),
        ])

# Model

In [ ]:
class EffNet(nn.Module):#EFFNET
    def __init__(self,MODEL_ARCH, num_classes, pretrained = False):
        super().__init__()

        self.model = timm.create_model(MODEL_ARCH,pretrained = pretrained )
        n_features = self.model.classifier.in_features
        self.model.global_pool = nn.Identity()
        self.model.classifier = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features,num_classes)
        self.soft = nn.Softmax(dim=1)
        
    def forward(self,x): 
        bs = x.size(0) # bs -> batch size
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs,-1)
        output = self.fc(pooled_features)
        output = self.soft(output)

        return output 
    
class ResNet200D(nn.Module):
    def __init__(self, MODEL_ARCH, num_classes, pretrained=False):
        super().__init__()
        
        self.model = timm.create_model(MODEL_ARCH, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, num_classes)
        self.soft = nn.Softmax(dim=1)
        
    def forward(self,x): 
        bs = x.size(0) # bs -> batch size
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs,-1)
        output = self.fc(pooled_features)
        output = self.soft(output)

        return output 

In [ ]:
def GetCriterion(criterion_name, criterion=None):

    if criterion_name == 'FocalLoss':
        criterion = FocalLoss()
    elif criterion_name == 'CustomLoss':
        criterion = CustomLoss(WEIGHTS)
    elif criterion_name == 'BCE':
        criterion = nn.BCEWithLogitsLoss()
    return criterion
    
    
def GetScheduler(scheduler_name, optimizer, batches=None):
    if scheduler_name == 'OneCycleLR':
        return torch.optim.lr_scheduler.OneCycleLR(optimizer,max_lr = 1e-2,epochs=EPOCHS,
                                                   steps_per_epoch = batches+1,pct_start = 0.1)
    if scheduler_name == 'CosineAnnealingWarmRestarts':
        return torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = T_0, T_mult=1,
                                                                    eta_min=MIN_LR, last_epoch=-1)
    elif scheduler_name == 'CosineAnnealingLR':
        return torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_MAX, eta_min=0, last_epoch=-1)
    elif scheduler_name == 'ReduceLROnPlateau':
        return torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=1, threshold=0.0001,
                                                          cooldown=0, min_lr=MIN_LR)
    
def GetOptimizer(optimizer_name,parameters):
    #['Adam','Ranger']
    if optimizer_name == 'Adam':
        return torch.optim.Adam(parameters, lr=LR, weight_decay=WEIGHT_DECAY, amsgrad=False)
    elif optimizer_name == 'AdamW':
        return torch.optim.Adam(parameters, lr=LR, weight_decay=WEIGHT_DECAY, amsgrad=False)
    elif optimizer_name == 'AdamP':
        return AdamP(parameters, lr=LR, weight_decay=WEIGHT_DECAY)
    elif optimizer_name == 'Ranger':
        return Ranger(parameters, lr = LR, alpha = 0.5, k = 6, N_sma_threshhold = 5, 
                      betas = (0.95,0.999), weight_decay=WEIGHT_DECAY)

# Train and Validation Functions

In [ ]:
def train_fn(model, dataloader, device, epoch, optimizer, criterion, scheduler):
    
    losses = AverageMeter()
    accuracies = AverageMeter()
    
    model.train()
    scaler = GradScaler()
    start_time = time.time()
    loader = tqdm(dataloader, total=len(dataloader))
    for step, (images, labels) in enumerate(loader):

        images = images.to(device).float()
        labels = labels.to(device).float()

        with autocast():
            logits = model(images)
            loss = criterion(logits, labels)
            losses.update(loss.item(), BATCH_SIZE)
    
            scaler.scale(loss).backward()
            grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm = MAX_NORM)

            if (step+1) % ITERS_TO_ACCUMULATE == 0:
                scaler.step(optimizer)
                # Update the scale for next iteration.
                scaler.update()
                optimizer.zero_grad()
        
        if scheduler is not None and SCHEDULER_UPDATE == 'batch':
            scheduler.step()

        start_time = time.time()
        
        if step % ITER_FREQ == 0:
            
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Loss: {loss.val:.4f} ({loss.avg:.4f})'.format((epoch+1),
                                                                  step, len(dataloader),
                                                                  loss=losses))

        loader.set_description(f'Training Epoch {epoch+1}/{EPOCHS}')
        loader.set_postfix(loss=losses.avg)
#         del images, labels
    if scheduler is not None and SCHEDULER_UPDATE == 'epoch':
        scheduler.step()
        
    return losses.avg

In [ ]:
def valid_fn(epoch, model, criterion, val_loader, device, scheduler):
    
    model.eval()
    losses = AverageMeter()
    accuracies = AverageMeter()
    PREDS = []
    TARGETS = []
    loader = tqdm(val_loader, total=len(val_loader))
    with torch.no_grad():  # without torch.no_grad() will make the CUDA run OOM.
        for step, (images, labels) in enumerate(loader):

            images = images.to(device)
            labels = labels.to(device).float()

            output = model(images)
            loss = criterion(output, labels)
            losses.update(loss.item(), BATCH_SIZE)
            PREDS += [output.sigmoid()]
            TARGETS += [labels.detach().cpu()]
            loader.set_description(f'Validating Epoch {epoch+1}/{EPOCHS}')
            loader.set_postfix(loss=losses.avg)
#             del images, labels
    PREDS = torch.cat(PREDS).cpu().numpy()
    TARGETS = torch.cat(TARGETS).cpu().numpy()
    roc_auc = macro_multilabel_auc(TARGETS, PREDS)
    if scheduler is not None:
        scheduler.step()
        
    return losses.avg, roc_auc

# Engine and Main

In [ ]:
def engine(device, folds, fold, model_path=None):

    train_data = SIIM_COVID(df[df['fold'] != fold] , train = True,
                                        transforms = get_transform(), IMG_SIZE = IMG_SIZE)
    val_data = SIIM_COVID(df[df['fold'] == fold] , train = True,# To pass both image and targets 
                                        transforms = get_transform(train=False), IMG_SIZE = IMG_SIZE)

    train_loader = DataLoader(train_data,
                              batch_size=BATCH_SIZE, 
                              shuffle=False, 
                              num_workers=NUM_WORKERS,
                              pin_memory=True, # enables faster data transfer to CUDA-enabled GPUs.
                              drop_last=True)
    val_loader = DataLoader(val_data, 
                            batch_size=VAL_BATCH_SIZE,
                            num_workers=NUM_WORKERS,
                            shuffle=False, 
                            pin_memory=True,
                            drop_last=False)
    
    if model_path is not None:
        model = EffNet(MODEL_ARCH, 4, pretrained=False)
        model.load_state_dict(torch.load(model_path))
#         START_EPOCH = int(model_path.split('_')[-2])
    else:
        model = EffNet(MODEL_ARCH, 4, pretrained=True)

        START_EPOCH = 0
    model.to(device)
    
    params = filter(lambda p: p.requires_grad, model.parameters())    
    optimizer = GetOptimizer(OPTIMIZER, params)
    
    train_criterion = GetCriterion(TR_CRITERION).to(device)    
    val_criterion = GetCriterion(VL_CRITERION).to(device)
    
    scheduler = GetScheduler(SCHEDULER, optimizer)
    
    loss = []
    for epoch in range(START_EPOCH, EPOCHS):
        
        epoch_start = time.time()        
        avg_loss = train_fn(model, train_loader, device, epoch, optimizer, train_criterion, scheduler)

        torch.cuda.empty_cache()
        avg_val_loss, roc_auc_score  = valid_fn(epoch, model, val_criterion, val_loader, device, scheduler)
        epoch_end = time.time() - epoch_start
        
        print(f'Training Loss after epoch {epoch+1}: {avg_loss:.4f}')
        print(f'Validation Loss after epoch {epoch+1}: {avg_val_loss:.4f}')
        print(f'Validation ROC AUC Score after epoch {epoch+1}: {roc_auc_score:.4f}')
        print(f'Epoch {epoch+1} finished in {epoch_end:.0f}s')
        loss.append(avg_loss)
        
        content = f'Fold {fold} Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f} roc_auc_score: {roc_auc_score :.4f} time: {epoch_end:.0f}s'
        with open(f'{MODEL_ARCH}_{OPTIMIZER}_{IMG_SIZE}.txt', 'a') as appender:
            appender.write(content + '\n')                                         

        PATH = f'{MODEL_ARCH}_fold_{fold}_epoch_{(epoch+1)}_{round(roc_auc_score,4)*100}.pt'
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_val_loss,
            }, PATH)
        torch.cuda.empty_cache()
    
    return loss

In [ ]:
if __name__ == '__main__':

    df = stratifiedKFold(df = df,num_folds = N_FOLDS,random_state = 42)
    for fold in range(START_FOLD, N_FOLDS):
        if fold == 4:
            break
        print(f'===== Fold {fold} Starting =====')
        fold_start = time.time()
        logs = engine(DEVICE, df, fold, MODEL_PATH)
        print(f'Time taken in fold {fold}: {time.time()-fold_start}')